# $\mathcal M$-Parallel Classes

The purpose of this notebook is to use Sage to assist in the classification of rank 2 invariant subvarieties in strata of translation surfaces in genus 3. This classification was already done through other methods by David Aulicino, Duc Man Nguyen, and Alex Wright, but the methods described here, which is also used in my forthcoming PhD thesis will useful in how it can be readily applied to other strata.

Assume $\mathcal M$ is an invariant subvariety in a stratum $\mathcal H$. We will start with a horizontally periodic translation surface $M\in \mathcal M$. A database for the possible cylinder diagrams for $M$ can be found in `surface-dynamics`: see [here](https://flatsurf.github.io/surface-dynamics/database.html). Here is an example:

In [1]:
from surface_dynamics import CylinderDiagram

cd = CylinderDiagram('(0,2)-(6) (1,4)-(3,5) (3,6)-(2,4) (5)-(0,1)')
cd

(0,2)-(6) (1,4)-(3,5) (3,6)-(2,4) (5)-(0,1)

![Drawing of above cylinder diagram](h211-c4-9-1.png)

Assume $M$ has a cylinder diagram with $n$ cylinders, and label them $0$ to $n-1$. Assume that these cylinders are partitioned into $m$ distinct $\cal M$-parallel classes, and we want to know all possible ways to do this. The following function uses existing Sage functions to list all possible partitions. The point of this notebook is to filter out some of these candidate partitions. Although we won't use it here, you can set `singletons=False` if you do not want to allow cylinders in an $\cal M$-parallel class by themselves.

In [2]:
from sage.all import Partitions, SetPartitions

def list_partitions(n, m, singletons=True):
    """Return a list of all ways to partition the set [1..n] into m sets.
    
    If singletons==False, do not allow singleton sets.

    Each element of each partition is a frozen set.
    Each partition is an instance of
    `sage.combinat.set_partition.SetPartitions_setparts_with_category.element_class`."""

    partitions = []

    # Partition the integer n into m nonzero integers
    int_parts = Partitions(n, length=m).list()

    # Checks for singleton sets
    if not singletons:
        int_parts = [l for l in int_parts if not any([i == 1 for i in l])]
    
    # Coverts integer partitions into set partitions
    for each_part in int_parts:
        partitions.extend(SetPartitions(range(n), each_part))
    return partitions

# Tests
assert len(list_partitions(5, 2, singletons=True)) == 15
assert len(list_partitions(6, 2, singletons=False)) == 25
assert len(list_partitions(6, 3, singletons=False)) == 15

list_partitions(4, 2)

[{{0}, {1, 2, 3}},
 {{0, 1, 2}, {3}},
 {{0, 1, 3}, {2}},
 {{0, 2, 3}, {1}},
 {{0, 1}, {2, 3}},
 {{0, 3}, {1, 2}},
 {{0, 2}, {1, 3}}]

In [3]:
def find_cylinder_in_partition(partition, cylinder):
    """Find the M-parallel class in `partition` than contains `cylinder`.
    
    `partition` is a list of frozen sets.
    `cylinder` is an integer
    Return the index of the element of `partition` that contains `cylinder`."""
    for i, parallel_class in enumerate(partition):
        if cylinder in parallel_class:
            return i
    
    # If cylinder was not found in partition
    return None

def check_pants_condition(partition, pants_list):
    """Check the partition satisfies any homology conditions coming from 
    the pants in pants_list.
    
    `partition` is a partition of the horizontal cylinders into M-parallel
    classes.

    `pants_list` is a list of pants, where each pants is a frozenset containing
    every cylinder in the pants
    
    The homology condition is the following:
    the cylinders in the pants cannot be contained in exactly two distinct sets
    in `partition`.
    """

    for pants in pants_list:
        
        # A list of the cylinder classes that contain a cylinder of pants
        cylinder_classes = map(
            lambda c: find_cylinder_in_partition(partition, c),
            pants
        )
        if len(set(cylinder_classes)) == 2:
            return False
    return True

def find_generic_pants(cyl_graph):
        """Finds cases when n cylinders are all only attached to the side
        of a single cylinder. This is like a generalized version of a
        topological pair of pants allows n pant legs.
        
        Input: A cylinder diagram.
        
        Output: A set of pants. Each pants is a frozenset consisting of the 
        cylinders in the pants.
        
        Note: For Python 3.7 and higher, frozensets should maintain insertion
        order. Thus, the first element of each pants is the waist curve and the
        rest are the pant legs. However, we do not use this in our code."""

        pants_set = set()
        for n in cyl_graph:
            # If every cylinder above `C` is only adjacent to `C` along its
            # bottom, this is a generic pants.
            successors = list(self.digraph.successors(n))
            if all([list(self.digraph.predecessors(suc)) == [n] 
                    for suc in successors]):
                
                pants = frozenset([n] + successors)
                pants_set.add(pants)

            # If every cylinder below `C` is only adjacent to `C` along its
            # top, this is a generic pants.
            predecessors = list(self.digraph.predecessors(n))
            if all([list(self.digraph.successors(pre)) == [n] 
                    for pre in predecessors]):

                pants = frozenset([n] + predecessors)
                pants_set.add(pants)
        return pants_set

def filter_pants_condition(cd, part_list):
    """Filter out the partitions in part_list when check_pants_condition=False.
    """
    cyl_graph = cd.cylinder_graph()
    pants_list = list(cyl_graph.find_generic_pants())
    return [partition for partition in part_list 
                      if check_pants_condition(partition, pants_list)]

cd = CylinderDiagram("(0,3)-(5) (1)-(2) (2,5)-(3,4) (4)-(0,1)")
partitions = list_partitions(4, 2)
filter_pants_condition(cd, partitions)

AttributeError: 'DiGraph' object has no attribute 'find_generic_pants'